## Web Scraper
We are scrapping links from direct atheletics (https://www.directathletics.com/rankings.html) to scrap data from the main athletic database (https://www.tfrrs.org/) 

In [1]:
# Download all necessary libraries
%pip install requests beautifulsoup4 pandas numpy matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Python\python.exe -m pip install --upgrade pip' command.


In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd 

# URL Link for Team Stats 
direct_athletics_url = 'https://www.directathletics.com/rankings.html'

def get_team_links(html_content, team)